In [25]:
# Imports
import pickle
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

In [26]:
# Train vs other
train = False
exploring = False

In [27]:
# Getting the data
df = pd.DataFrame()
if (train):
    df = pd.read_csv("data/lending_train.csv")
else:
    df = pd.read_csv("data/lending_topredict.csv")
pd.set_option('max_columns', None)

In [28]:
if (True):
    print(df.dtypes)
    print(df.describe())

ID                            int64
requested_amnt              float64
loan_duration                object
employment                   object
employment_length            object
race                         object
reason_for_loan              object
extended_reason              object
annual_income               float64
debt_to_income_ratio        float64
employment_verified          object
public_bankruptcies         float64
zipcode                      object
state                        object
home_ownership_status        object
delinquency_last_2yrs       float64
fico_score_range_low        float64
fico_score_range_high       float64
fico_inquired_last_6mths    float64
months_since_last_delinq    float64
revolving_balance           float64
total_revolving_limit       float64
type_of_application          object
any_tax_liens               float64
loan_paid                    object
dtype: object
                 ID  requested_amnt  annual_income  debt_to_income_ratio  \
count  3.4

In [29]:
# Removing bad cols
if (train):
    df = df.drop(columns=['ID', 'race', 'extended_reason'])
else:
    df = df.drop(columns=['race', 'extended_reason', 'loan_paid'])

In [30]:
# Finding num of bad vals per col
if (exploring):
    for col in df.columns:
        print(col, ":", df[col].isnull().sum())

In [31]:
# Finding correlation between # of missing values and loan_paid
if (exploring):
    missing_vals = df.shape[0] * [0]
    for index in range (df.shape[0]):
        missing_vals[index] = (df.iloc[index].isnull().sum())

    print(np.corrcoef(np.array(missing_vals).squeeze(axis=1), df['loan_paid'].to_numpy()))

In [32]:
# Creating new column containing # of missing values initially
missing_vals = df.shape[0] * [0]
for index in range (df.shape[0]):
        missing_vals[index] = (df.iloc[index].isnull().sum())
df['missing_vals'] = missing_vals

In [33]:
# Filling in bad values using median or mode
categorical_cols = ['employment_length', 'zipcode', 'state']
numerical_cols = ['debt_to_income_ratio', 'public_bankruptcies', 'fico_inquired_last_6mths', 'months_since_last_delinq', 'total_revolving_limit', 'any_tax_liens']
other_cols = ['employment', 'home_ownership_status']

for col in categorical_cols:
    mode = df[col].mode()[0]
    df[col].fillna(mode, inplace=True)
    df[col].replace([np.inf, -np.inf], mode, inplace=True)
    
for col in numerical_cols:
    median = df[col].median()
    df[col].fillna(median, inplace=True)
    df[col].replace([np.inf, -np.inf], median, inplace=True)
    
for col in other_cols:
    df[col].fillna('unknown', inplace=True)
    df[col].replace([np.inf, -np.inf], 'unknown', inplace=True)

df

,ID,requested_amnt,loan_duration,employment,employment_length,reason_for_loan,annual_income,debt_to_income_ratio,employment_verified,public_bankruptcies,zipcode,state,home_ownership_status,delinquency_last_2yrs,fico_score_range_low,fico_score_range_high,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,type_of_application,any_tax_liens,missing_vals
0,1000000,9600.0,36 months,Computer Technician,< 1 year,debt_consolidation,30000.0,19.01,Verified,0.0,465xx,IN,RENT,0.0,680.0,684.0,0.0,31.0,18492.0,22900.0,Individual,0.0,1
1,1000001,8000.0,36 months,Teacher,6 years,debt_consolidation,34500.0,31.30,Source Verified,0.0,941xx,CA,RENT,0.0,650.0,654.0,0.0,31.0,5119.0,26000.0,Individual,0.0,1
2,1000002,21000.0,36 months,Portability Specialist,10+ years,other,45000.0,13.15,Verified,1.0,330xx,FL,MORTGAGE,0.0,650.0,654.0,0.0,31.0,5579.0,8700.0,Individual,0.0,1
3,1000003,5000.0,36 months,Internal Maintence,2 years,credit_card,30000.0,6.74,Not Verified,0.0,864xx,AZ,RENT,0.0,650.0,654.0,1.0,31.0,3872.0,22100.0,Individual,0.0,1
4,1000004,3000.0,36 months,Office administration,< 1 year,other,38000.0,0.79,Source Verified,0.0,950xx,CA,OWN,0.0,650.0,654.0,0.0,31.0,250.0,8800.0,Individual,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345305,1345305,3000.0,36 months,pharmacy technician,< 1 year,debt_consolidation,24000.0,5.75,Source Verified,0.0,952xx,CA,OWN,0.0,650.0,654.0,0.0,28.0,3629.0,20400.0,Individual,0.0,0
345306,1345306,12000.0,36 months,Senior Account Manage,6 years,debt_consolidation,65000.0,19.31,Source Verified,0.0,142xx,NY,OWN,1.0,650.0,654.0,2.0,19.0,10010.0,18200.0,Individual,0.0,0
345307,1345307,30100.0,36 months,unknown,10+ years,debt_consolidation,87028.0,28.71,Verified,0.0,611xx,IL,MORTGAGE,2.0,665.0,669.0,1.0,9.0,7886.0,9400.0,Individual,0.0,2
345308,1345308,27000.0,36 months,Quality Assurance Associate,7 years,debt_consolidation,100400.0,21.22,Not Verified,0.0,913xx,CA,RENT,0.0,705.0,709.0,0.0,31.0,19109.0,32200.0,Individual,0.0,1


In [34]:
# Finding correlation between variables
if (exploring):
    cor_matrix = df.corr()
    plt.figure(figsize = (30, 15))
    sns.heatmap(cor_matrix, annot=True)

In [35]:
# Feature engineering

## generating new col 
income_to_loan_amount_ratio = df['requested_amnt'] / df['annual_income']

if (exploring):
    print("Income to loan paid correlation:", df['annual_income'].corr(df['loan_paid']))
    print("Requested amount to loan paid correlation:", df['requested_amnt'].corr(df['loan_paid']))
    print("Income to loan amount ratio to loan paid correlation:", income_to_loan_amount_ratio.corr(df['loan_paid']))

df['income_to_loan_amount_ratio'] = income_to_loan_amount_ratio

# combining fico high and low score
fico_score = (df['fico_score_range_high'] + df['fico_score_range_low']) / 2
fico_score_range = df['fico_score_range_high'] - df['fico_score_range_low']

df.drop(inplace=True, columns=['fico_score_range_high', 'fico_score_range_low'])

df['fico_score'] = fico_score
df['fico_score_range'] = fico_score_range

In [36]:
# Removing unnecessary strings
new_loan_duration = []
new_employment_length = []

for index in range (df.shape[0]):
    new_loan_duration.append(df['loan_duration'].iloc[index].replace(" months", ""))
    new_employment_length.append(df['employment_length'].iloc[index].replace(" years", "").replace("< 1 year", ".5").replace(" year", "").replace("+", "")
)

df.drop(inplace=True, columns=['loan_duration', 'employment_length'])
df['loan_duration'] = new_loan_duration
df['employment_length'] = new_employment_length

In [37]:
# analyzing home ownsership data to find ordinality
if (exploring):
    home_ownerships_status_vals = set(df['home_ownership_status'])
    for home_ownerships_status_val in home_ownerships_status_vals:
        subset = df[df['home_ownership_status'] == home_ownerships_status_val]['loan_paid']
        print(home_ownerships_status_val, "\tNum of Examples:", subset.shape[0], "\tMean:", subset.mean())

# not very fruitful

In [38]:
# encoding based on data ordinality
encodings = {'employment_verified': {'Not Verified': 0, 'Verified': 1, 'Source Verified': 2},
             'type_of_application': {'Individual': 0, 'Joint App': 1},
             'loan_duration': {'36': 0, ' 36': 0, '60': 1, ' 60': 1}}
for col in encodings.keys():
    new_col = []
    for item in df[col]:
        new_col.append(encodings[col][item])
    df = df.drop(columns=[col])
    df[col] = new_col

In [39]:
# special encoding for cols with too many unqiue values
if (exploring):
    top_five_employment = df['employment'].value_counts()
    top_five_reason = df['reason_for_loan'].value_counts()
    print("Top 5 Most Common Jobs", top_five_employment, '\n')
    print("Top 5 Most Common Reasons for Jobs", top_five_reason, '\n')

top_five_employment = ['Teacher', 'Manager', 'Owner', 'Registered Nurse', 'RN']
top_five_reason = ['debt_consolidation', 'credit_card', 'home_improvement', 'major_purchase', 'small_business']
        
new_employment_list = df.shape[0] * [0]
new_reason_list = df.shape[0] * [0]
for index in range (df.shape[0]):
    if df['employment'].iloc[index] in top_five_employment:
        new_employment_list[index] = df['employment'].iloc[index]
    else:
        new_employment_list[index] = 'other'
        
    if df['reason_for_loan'].iloc[index] in top_five_reason:
        new_reason_list[index] = df['reason_for_loan'].iloc[index]
    else:
        new_reason_list[index] = 'other'
    
df.drop(columns=['employment', 'reason_for_loan'], inplace=True)
df['employment'] = new_employment_list
df['reason_for_loan'] = new_reason_list

In [40]:
# giving encodings to states based on their ordinality
ordering = ['MS', 'IA', 'NE', 'AR', 'AL', 'OK', 'LA', 'NY', 'NV', 'TN', 'IN', 'MO', 'NM', 'MD', 'SD', 'FL', 'NJ', 'PA', 'KY', 'NC', 'OH', 'ND', 'MI', 'VA', 'HI', 'TX', 'AZ', 'MN', 'AK', 'CA', 'ID', 'DE', 'MA', 'GA', 'IL', 'WI', 'RI', 'CT', 'KS', 'UT', 'WY', 'SC', 'MT', 'CO', 'WV', 'WA', 'NH', 'OR', 'VT', 'ME', 'DC']

if (exploring):
    values = set(df['state'])
    values_avg = {}
    for value in values:
        values_avg[value] = df[df['state'] == value]['loan_paid'].mean()

    print(values_avg)
    x = dict(sorted(values_avg.items(), key=lambda item: item[1]))
    print(x)
    print(list(x.keys()))
        
new_states_col = df.shape[0] * [0]
for index in range (df.shape[0]):
    new_states_col[index] = ordering.index(df['state'].iloc[index])
    
df['state'] = new_states_col

In [41]:
df

,ID,requested_amnt,annual_income,debt_to_income_ratio,public_bankruptcies,zipcode,state,home_ownership_status,delinquency_last_2yrs,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,any_tax_liens,missing_vals,income_to_loan_amount_ratio,fico_score,fico_score_range,employment_length,employment_verified,type_of_application,loan_duration,employment,reason_for_loan
0,1000000,9600.0,30000.0,19.01,0.0,465xx,10,RENT,0.0,0.0,31.0,18492.0,22900.0,0.0,1,0.320000,682.0,4.0,.5,1,0,0,other,debt_consolidation
1,1000001,8000.0,34500.0,31.30,0.0,941xx,29,RENT,0.0,0.0,31.0,5119.0,26000.0,0.0,1,0.231884,652.0,4.0,6,2,0,0,Teacher,debt_consolidation
2,1000002,21000.0,45000.0,13.15,1.0,330xx,15,MORTGAGE,0.0,0.0,31.0,5579.0,8700.0,0.0,1,0.466667,652.0,4.0,10,1,0,0,other,other
3,1000003,5000.0,30000.0,6.74,0.0,864xx,26,RENT,0.0,1.0,31.0,3872.0,22100.0,0.0,1,0.166667,652.0,4.0,2,0,0,0,other,credit_card
4,1000004,3000.0,38000.0,0.79,0.0,950xx,29,OWN,0.0,0.0,31.0,250.0,8800.0,0.0,1,0.078947,652.0,4.0,.5,2,0,0,other,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345305,1345305,3000.0,24000.0,5.75,0.0,952xx,29,OWN,0.0,0.0,28.0,3629.0,20400.0,0.0,0,0.125000,652.0,4.0,.5,2,0,0,other,debt_consolidation
345306,1345306,12000.0,65000.0,19.31,0.0,142xx,7,OWN,1.0,2.0,19.0,10010.0,18200.0,0.0,0,0.184615,652.0,4.0,6,2,0,0,other,debt_consolidation
345307,1345307,30100.0,87028.0,28.71,0.0,611xx,34,MORTGAGE,2.0,1.0,9.0,7886.0,9400.0,0.0,2,0.345866,667.0,4.0,10,1,0,0,other,debt_consolidation
345308,1345308,27000.0,100400.0,21.22,0.0,913xx,29,RENT,0.0,0.0,31.0,19109.0,32200.0,0.0,1,0.268924,707.0,4.0,7,0,0,0,other,debt_consolidation


In [42]:
# simplifying categories for zipcode based on correlation with loan paid
cols = ['zipcode']
if (train):
    for col in cols:
        values = set(df[col])
        values_avg = {}
        for value in values:
            values_avg[value] = df[df[col] == value]['loan_paid'].mean()

        summary_df = pd.DataFrame(values_avg.values())
        percentile_20 = summary_df.quantile(.2)[0]
        percentile_40 = summary_df.quantile(.4)[0]
        percentile_60 = summary_df.quantile(.6)[0]
        percentile_80 = summary_df.quantile(.8)[0]
        
        col_classification = {}
        for value in values:
            avg = values_avg[value]
            if (avg < percentile_20):
                col_classification[value] = 'bad'
            elif (avg < percentile_40):
                col_classification[value] = 'mid_bad'
            elif (avg < percentile_60):
                col_classification[value] = 'mid'
            elif (avg < percentile_80):
                col_classification[value] = 'mid_good'
            else:
                col_classification[value] = 'good'
        
        with open('data/' + col + '.pickle', 'wb') as handle:
            pickle.dump(col_classification, handle, protocol=pickle.HIGHEST_PROTOCOL)

        new_col = df.shape[0] * [0]
        for index in range (df.shape[0]):
            new_col[index] = col_classification[df[col].iloc[index]]

        df.drop(columns=[col])
        df[col] = new_col
else:
    for col in cols:
        with open('data/' + col + '.pickle', 'rb') as handle:
            col_classification = pickle.load(handle)

        new_col = df.shape[0] * [0]
        for index in range (df.shape[0]):
            try:
                new_col[index] = col_classification[df[col].iloc[index]]
            except:
                new_col[index] = 'mid'
                
        df.drop(columns=[col])
        df[col] = new_col

In [43]:
# one-hot encoding necessary columns
cols_to_one_hot_encode = ['reason_for_loan', 'employment', 'home_ownership_status', 'zipcode']
for col in cols_to_one_hot_encode:
    df = pd.get_dummies(df, columns=[col])

In [44]:
# fixing ordinality of months_since_last_delinq
fixed_months_since_last_delinq = df.shape[0] * [0]
for index in range (df.shape[0]):
    if (df['months_since_last_delinq'].iloc[index] == 0):
        fixed_months_since_last_delinq[index] = 250
    else:
        fixed_months_since_last_delinq[index] = df['months_since_last_delinq'].iloc[index]
        
df['months_since_last_delinq'] = fixed_months_since_last_delinq

In [45]:
# normalizing columns and filling bad values
columns_to_norm = ['requested_amnt', 'annual_income', 'debt_to_income_ratio', 'months_since_last_delinq', 'fico_score', 'revolving_balance', 'total_revolving_limit']

if (train):
    train_attrs = {}
    for col in columns_to_norm:
        train_attrs[col] = (df[col].mean(), df[col].std())
        df[col] = (df[col] - train_attrs[col][0]) / train_attrs[col][1]
    with open('data/train_attributes.pickle', 'wb') as handle:
        pickle.dump(train_attrs, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    train_attrs = {}
    with open('data/train_attributes.pickle', 'rb') as handle:
        train_attrs = pickle.load(handle)
    for col in columns_to_norm:
        df[col] = (df[col] - train_attrs[col][0]) / train_attrs[col][1]

In [46]:
# Converting floats to ints to save space
cols = ['delinquency_last_2yrs', 'fico_inquired_last_6mths', 'public_bankruptcies', 'any_tax_liens', 'fico_score_range']
for col in cols:
    df[col] = df[col].astype(int)

In [47]:
# converting from float64 to float32 to save space
for col in df.columns:
    if (df[col].dtype == 'float64'):
        df[col] = df[col].astype('float32')

In [48]:
# Write to csv
if (train):
    df.to_csv('data/train_preprocessed_data.csv', index=None)
else:
    df.to_csv('data/predict_preprocessed_data.csv', index=None)
df

,ID,requested_amnt,annual_income,debt_to_income_ratio,public_bankruptcies,state,delinquency_last_2yrs,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,any_tax_liens,missing_vals,income_to_loan_amount_ratio,fico_score,fico_score_range,employment_length,employment_verified,type_of_application,loan_duration,reason_for_loan_credit_card,reason_for_loan_debt_consolidation,reason_for_loan_home_improvement,reason_for_loan_major_purchase,reason_for_loan_other,reason_for_loan_small_business,employment_Manager,employment_Owner,employment_RN,employment_Registered Nurse,employment_Teacher,employment_other,home_ownership_status_ANY,home_ownership_status_MORTGAGE,home_ownership_status_NONE,home_ownership_status_OTHER,home_ownership_status_OWN,home_ownership_status_RENT,zipcode_bad,zipcode_good,zipcode_mid,zipcode_mid_bad,zipcode_mid_good
0,1000000,-0.553173,-0.655125,0.065823,0,10,0,0,-0.113201,0.099462,-0.256915,0,1,0.320000,-0.297978,4,.5,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,1000001,-0.736681,-0.591423,1.168160,0,29,0,0,-0.113201,-0.494430,-0.172661,0,1,0.231884,-1.194323,4,6,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0
2,1000002,0.754322,-0.442785,-0.459782,1,15,0,0,-0.113201,-0.474001,-0.642853,0,1,0.466667,-1.194323,4,10,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
3,1000003,-1.080758,-0.655125,-1.034719,0,26,0,1,-0.113201,-0.549809,-0.278658,0,1,0.166667,-1.194323,4,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
4,1000004,-1.310143,-0.541877,-1.568397,0,29,0,0,-0.113201,-0.710661,-0.640135,0,1,0.078947,-1.194323,4,.5,2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345305,1345305,-1.310143,-0.740061,-1.123516,0,29,0,0,-0.286074,-0.560600,-0.324862,0,0,0.125000,-1.194323,4,.5,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
345306,1345306,-0.277911,-0.159665,0.092731,0,7,1,2,-0.804692,-0.277222,-0.384655,0,0,0.184615,-1.194323,4,6,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
345307,1345307,1.798023,0.152163,0.935853,0,34,2,1,-1.380935,-0.371548,-0.623828,0,2,0.345866,-0.746151,4,10,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
345308,1345308,1.442476,0.341457,0.264047,0,29,0,0,-0.113201,0.126863,-0.004153,0,1,0.268924,0.448976,4,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0


In [49]:
df.shape

(345310, 43)